In [1]:
# Import this library
from nbacktest import Backtest, Strategy, RealBroker

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
universe = ["COGN3.SA", "BBDC3.SA"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2022-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

# Rename ticker level (level=1)
df_ohlc = df_ohlc.rename(
    columns=lambda x: x.replace(".SA", ""),
    level=1
)

universe = ["COGN3", "BBDC3"] 

df_ohlc = df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(250, 10)


In [3]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = 100

        order1 = self.buy("BBDC3", quantity1)

        trade1 = self.create_trade(orders=[order1], notes="Real Trade Example")

        print("Bought %s" % (quantity1))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass

In [4]:
df_ohlc

Price           Close                 High                  Low            \
Ticker          BBDC3     COGN3      BBDC3     COGN3      BBDC3     COGN3   
Date                                                                        
2022-12-23  10.209590  2.112774  10.255442  2.141983  10.049110  2.044620   
2022-12-26   9.972689  2.103038  10.156094  2.132246   9.942121  2.054356   
2022-12-27  10.033823  2.005675  10.186661  2.112774   9.903911  2.005675   
2022-12-28  10.339499  2.103038  10.377708  2.112774  10.026180  2.005675   
2022-12-29  10.293648  2.064092  10.469411  2.141983  10.270722  2.034883   

Price            Open              Volume            
Ticker          BBDC3     COGN3     BBDC3     COGN3  
Date                                                 
2022-12-23  10.117887  2.064092   3177800  21629400  
2022-12-26  10.110243  2.122510   2063500   5967100  
2022-12-27  10.186661  2.112774  26504100  16932500  
2022-12-28  10.087316  2.015411  23236400  15476500  
2022-12-29  10.370066  2.112774  14129200  20212700

In [5]:
from nbacktest.core.broker import RealBroker

base_url = "http://metatrader_api_demo:8080/"
real_broker = RealBroker(universe=universe, cash=10_000, base_url=base_url)

bt = Backtest(data=df_ohlc,
                   universe=universe,
                   strategy_class=TestStrategy,
                   price_column="Close",
                   cash=10_000,
                   broker=real_broker)

result = bt.run()
result.head()

Strategy started!
Bought 100
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2022-12-23,8409.0,1020.958996,0,9429.958996
2022-12-26,8409.0,997.268867,0,9406.268867
2022-12-27,8409.0,1003.382301,0,9412.382301
2022-12-28,8409.0,1033.949852,0,9442.949852
2022-12-29,8409.0,1029.364777,0,9438.364777


In [8]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,AGE,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,3bf1bcbb-a844-4fcd-8aa9-ed5959a74ccf,OPEN,Real Trade Example,1029.364777,2022-12-23,None,6 days,None,"{'BBDC3': {'quantity': 100.0, 'value': 1029.36...",1029.364777,{},0,None,None,None


In [9]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,NOTES,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED,PROVIDER_ID
0,c5fa7ad9-851a-41af-997d-fb6d197b254b,BBDC3,BUY,FILLED,,3bf1bcbb-a844-4fcd-8aa9-ed5959a74ccf,2022-12-23,100,15.91,0,-1591.0,-1591.0,2022-12-23,100.0,15.91,0.0,-1591.0,-1591.0,2300133647
